In [1]:
using Plots
using CSV
using DataFrames
using Statistics
using StatsPlots
using Printf
using LinearAlgebra

# Store Data
using DelimitedFiles

$$
D = \frac{mean(dx_n^2) + mean(dy_n^2)}{4\times Times}
$$

In [2]:
function plotData(path, methods, savepath, print) # Loads Data, alpha is horizontal beta is vertical
    println("All scale into nm by times to 1e18")

    # Initiation
    alpha_max = 0.8
    alpha_min = 0.0
    alpha_step = 0.2
    alpha_nStep = size(alpha_min:alpha_step:alpha_max, 1)
    beta_max = 0.8
    beta_min = 0.0
    beta_step = 0.2
    beta_nStep = size(beta_min:beta_step:beta_max, 1)
    
    raw = Matrix{Matrix{BigFloat}}(undef, beta_nStep, alpha_nStep)

    # Load data
    for b in 1:beta_nStep
        for a in 1:alpha_nStep
            alpha = round((a-1) * alpha_step; digits = 1)
            beta = round((b-1) * beta_step; digits = 1)
            raw[b, a] = Matrix{BigFloat}(DataFrame(CSV.File("$path$methods-data-alpha=$alpha-beta=$beta.csv", header=false)))
        end
    end
    
    time_step = size(raw[1, 1], 1)
    Times = 0.2*time_step
    
    # Calculating Diffusion Coefficient
    DiffusionCoefficient = Matrix{BigFloat}(undef, beta_nStep, alpha_nStep)
    
    for b in 1:beta_nStep
        for a in 1:alpha_nStep
            DiffusionCoefficient[b, a] = (mean(raw[b, a][:,1].^2) + mean(raw[b, a][:,2].^2)) / (4 * Times)
        end
    end

    # Warning: scale by 1e18
    DiffusionCoefficient = DiffusionCoefficient .* 1e18
    
    writedlm("$savepath$methods-analysis-DiffusionCoefficient-nm.csv", DiffusionCoefficient, ',', header=true);
    if print
        df_D = DataFrame(DiffusionCoefficient, ["alpha=$i" for i in alpha_min:alpha_step:alpha_max])
        df_D[!, :beta] = ["beta=$i" for i in beta_min:beta_step:beta_max]
        @show df_D
    end
    
    p1 = plot(grid=true, legend=true, title="", xlab = "Alpha Ratio", ylab = "Diffusion Coefficient (nm)")
    for b in 1:beta_nStep
        beta = round((b - 1) * beta_step, digits=1)
        plot!([i for i in alpha_min:alpha_step:alpha_max], DiffusionCoefficient[b, :], label="beta=$beta")
    end
    savefig(p1, "$savepath$methods-analysis-LineBeta-nm.png")
    
    p2 = plot(grid=true, legend=true, title="", xlab = "Beta Ratio", ylab = "Diffusion Coefficient (nm)")
    for a in 1:alpha_nStep
        alpha = round((a - 1) * alpha_step, digits=1)
        plot!([i for i in beta_min:beta_step:beta_max], DiffusionCoefficient[:, a], label="alpha=$alpha")
    end
    savefig(p2, "$savepath$methods-analysis-LineAlpha-nm.png")
    
    p3 = heatmap(["alpha=$i" for i in alpha_min:alpha_step:alpha_max], ["beta=$i" for i in beta_max:-beta_step:beta_min], DiffusionCoefficient, xlabel="HA Inactive Ratio", ylabel="NA Inactive Ratio", title="", c=:viridis, color=:auto)
    savefig(p3, "$savepath$methods-analysis-DiffusionCoefficient-HeatMap-nm.png")

    # Calculating Phi
    Phi_raw = Matrix{Vector{BigFloat}}(undef, beta_nStep, alpha_nStep) # matrix of phi vectors
    Phi_mean = Matrix{BigFloat}(undef, beta_nStep, alpha_nStep) # matrix of mean phi
    Phi_var = Matrix{BigFloat}(undef, beta_nStep, alpha_nStep) # matrix of mean phi
    for b in 1:beta_nStep
        for a in 1:alpha_nStep
            phi = Vector{BigFloat}(undef, (time_step-1)) # vectors of phi
            for n in 2:time_step
                top = raw[b, a][n,2] - raw[b, a][(n-1),2]
                bottom = raw[b, a][n,1] - raw[b, a][(n-1),1]
                phi[n-1] = atan(top/bottom)
            end
            Phi_raw[b, a] = phi
            Phi_mean[b, a] = mean(phi)
            Phi_var[b, a] = var(phi)
        end
    end

    writedlm("$savepath$methods-analysis-PhiMean-radius.csv", Phi_mean, ',', header=true);
    if print
        df_Phi_mean = DataFrame(Phi_mean, ["alpha=$i" for i in alpha_min:alpha_step:alpha_max])
        df_Phi_mean[!, :beta] = ["beta=$i" for i in beta_min:beta_step:beta_max]
        @show df_Phi_mean
    end
    
    writedlm("$savepath$methods-analysis-PhiVar-radius.csv", Phi_var, ',', header=true);
    if print
        df_Phi_var = DataFrame(Phi_var, ["alpha=$i" for i in alpha_min:alpha_step:alpha_max])
        df_Phi_var[!, :beta] = ["beta=$i" for i in beta_min:beta_step:beta_max]
        @show df_Phi_var
    end
    
    p4 = heatmap(["alpha=$i" for i in alpha_min:alpha_step:alpha_max], ["beta=$i" for i in beta_max:-beta_step:beta_min], Phi_mean, xlabel="HA Inactive Ratio", ylabel="NA Inactive Ratio", title="", c=:viridis, color=:auto)
    p5 = heatmap(["alpha=$i" for i in alpha_min:alpha_step:alpha_max], ["beta=$i" for i in beta_max:-beta_step:beta_min], Phi_var, xlabel="HA Inactive Ratio", ylabel="NA Inactive Ratio", title="", c=:viridis, color=:auto)
    savefig(p4, "$savepath$methods-analysis-PhiMean-HeatMap-radius.png")
    savefig(p5, "$savepath$methods-analysis-PhiVar-HeatMap-radius.png")

    # Calculating Absolute Mean Error (AME)
    AME_raw = Matrix{Vector{BigFloat}}(undef, beta_nStep, alpha_nStep) # matrix of phi vectors
    AME_mean = Matrix{BigFloat}(undef, beta_nStep, alpha_nStep) # matrix of mean phi
    AME_var = Matrix{BigFloat}(undef, beta_nStep, alpha_nStep) # matrix of mean phi
    for b in 1:beta_nStep
        for a in 1:alpha_nStep
            Phi = Phi_raw[b, a]
            Phi = pushfirst!(Phi, 0.0)
            AME = abs.(Phi - cumsum(raw[b, a][:, 3]))
            AME_raw[b, a] = AME
            AME_mean[b, a] = mean(AME)
            AME_var[b, a] = var(AME)
        end
    end

    writedlm("$savepath$methods-analysis-AMEMean-radius.csv", AME_mean, ',', header=true);
    if print
        df_AME_mean = DataFrame(AME_mean, ["alpha=$i" for i in alpha_min:alpha_step:alpha_max])
        df_AME_mean[!, :beta] = ["beta=$i" for i in beta_min:beta_step:beta_max]
        @show df_AME_mean
    end
    
    writedlm("$savepath$methods-analysis-AMEVar-radius.csv", AME_var, ',', header=true);
    if print
        df_AME_var = DataFrame(AME_var, ["alpha=$i" for i in alpha_min:alpha_step:alpha_max])
        df_AME_var[!, :beta] = ["beta=$i" for i in beta_min:beta_step:beta_max]
        @show df_AME_var
    end
    
    p6 = heatmap(["alpha=$i" for i in alpha_min:alpha_step:alpha_max], ["beta=$i" for i in beta_max:-beta_step:beta_min], AME_mean, xlabel="HA Inactive Ratio", ylabel="NA Inactive Ratio", title="", c=:viridis, color=:auto)
    p7 = heatmap(["alpha=$i" for i in alpha_min:alpha_step:alpha_max], ["beta=$i" for i in beta_max:-beta_step:beta_min], AME_var, xlabel="HA Inactive Ratio", ylabel="NA Inactive Ratio", title="", c=:viridis, color=:auto)
    savefig(p6, "$savepath$methods-analysis-AMEMean-HeatMap-radius.png")
    savefig(p7, "$savepath$methods-analysis-AMEVar-HeatMap-radius.png")

    # display(p1)
    # display(p2)
    # display(p3)
    # display(p4)
    # display(p5)
    # display(p6)
    # display(p7)
end

plotData (generic function with 1 method)

In [3]:
plotData("data/VDisjoint2000/", "VDisjoint", "Analysis/", false)

All scale into nm by times to 1e18


"C:\\Users\\yuang\\temp\\rex_iav_diffusion\\Analysis\\VDisjoint-analysis-AMEVar-HeatMap-radius.png"

In [4]:
plotData("data/Mixing1000/", "Mixing", "Analysis/", false)

All scale into nm by times to 1e18


"C:\\Users\\yuang\\temp\\rex_iav_diffusion\\Analysis\\Mixing-analysis-AMEVar-HeatMap-radius.png"

In [5]:
plotData("data/Random180/", "Random", "Analysis/", false)

All scale into nm by times to 1e18


"C:\\Users\\yuang\\temp\\rex_iav_diffusion\\Analysis\\Random-analysis-AMEVar-HeatMap-radius.png"